In [25]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

df = pd.read_csv('input/drug_purchases.csv', parse_dates=['purchase_date'])

display(df.dtypes)
display(df.head())

id                            int64
log_id                        int64
layout_id                     int64
staging_record_id             int64
created_by                   object
created_at                   object
modified_by                  object
modified_at                  object
deleted                        bool
address                      object
age                           int64
city                         object
lattitude                   float64
longitude                   float64
ndc11code                     int64
purchase_date        datetime64[ns]
unit_price                  float64
drug_name                    object
drug_class                   object
state                        object
drug_indication              object
person_code                   int64
pharmacy                     object
zip                          object
dtype: object

,id,log_id,layout_id,staging_record_id,created_by,created_at,modified_by,modified_at,deleted,address,...,ndc11code,purchase_date,unit_price,drug_name,drug_class,state,drug_indication,person_code,pharmacy,zip
0,410,46,87,595,SYSTEM,2018-10-08 21:30:06,SYSTEM,2018-10-08 21:30:06,False,5208 98TH ST,...,93001298,2016-01-05,0.1327,PANTOPRAZOLE TAB 40MG,ULCER DRUGS,TX,Ulcer,17546398,CVS PHARMACY,79424
1,2387,46,87,2612,SYSTEM,2018-10-08 21:30:27,SYSTEM,2018-10-08 21:30:27,False,251 N BROADWAY,...,54001729,2016-01-19,0.2450,PREDNISONE TAB 10MG,CORTICOSTEROIDS,NJ,Inflammation,17427509,SAVON PHARMACY,8070
2,2411,46,87,2587,SYSTEM,2018-10-08 21:30:28,SYSTEM,2018-10-08 21:30:28,False,2040 ROUTE 130,...,59762015502,2016-01-19,0.2734,ATORVASTATIN TAB 10MG,ANTIHYPERLIPIDEMICS,NJ,High Cholesterol,17112961,EXPRESS SCRIPTS,80169716
3,2436,46,87,2565,SYSTEM,2018-10-08 21:30:28,SYSTEM,2018-10-08 21:30:28,False,5230 DALLAS HWY,...,310080060,2016-01-18,304.5100,TUDORZA PRES AER 400/ACT,ANTIASTHMATIC AND BRONCHODILATOR AGENTS,GA,Asthma,17427513,WALGREENS #6325,30127
4,2461,46,87,2540,SYSTEM,2018-10-08 21:30:28,SYSTEM,2018-10-08 21:30:28,False,1704 W HILLSBOROUGH,...,59746028490,2016-01-18,0.1813,PANTOPRAZOLE TAB 40MG,ULCER DRUGS,FL,Ulcer,17106797,WALGREENS #6611,33603


In [50]:
display(df.shape)

df['drug_name'] = df['drug_name'].str.strip()
df.dropna(axis=0, subset=['person_code'], inplace=True)
df['person_code'] = df['person_code'].astype('str')
# df = df[~df['InvoiceNo'].str.contains('C')]
display(df.shape)
df.head()


(184250, 24)

(184250, 24)

,id,log_id,layout_id,staging_record_id,created_by,created_at,modified_by,modified_at,deleted,address,...,ndc11code,purchase_date,unit_price,drug_name,drug_class,state,drug_indication,person_code,pharmacy,zip
0,410,46,87,595,SYSTEM,2018-10-08 21:30:06,SYSTEM,2018-10-08 21:30:06,False,5208 98TH ST,...,93001298,2016-01-05,0.1327,PANTOPRAZOLE TAB 40MG,ULCER DRUGS,TX,Ulcer,17546398,CVS PHARMACY,79424
1,2387,46,87,2612,SYSTEM,2018-10-08 21:30:27,SYSTEM,2018-10-08 21:30:27,False,251 N BROADWAY,...,54001729,2016-01-19,0.2450,PREDNISONE TAB 10MG,CORTICOSTEROIDS,NJ,Inflammation,17427509,SAVON PHARMACY,8070
2,2411,46,87,2587,SYSTEM,2018-10-08 21:30:28,SYSTEM,2018-10-08 21:30:28,False,2040 ROUTE 130,...,59762015502,2016-01-19,0.2734,ATORVASTATIN TAB 10MG,ANTIHYPERLIPIDEMICS,NJ,High Cholesterol,17112961,EXPRESS SCRIPTS,80169716
3,2436,46,87,2565,SYSTEM,2018-10-08 21:30:28,SYSTEM,2018-10-08 21:30:28,False,5230 DALLAS HWY,...,310080060,2016-01-18,304.5100,TUDORZA PRES AER 400/ACT,ANTIASTHMATIC AND BRONCHODILATOR AGENTS,GA,Asthma,17427513,WALGREENS #6325,30127
4,2461,46,87,2540,SYSTEM,2018-10-08 21:30:28,SYSTEM,2018-10-08 21:30:28,False,1704 W HILLSBOROUGH,...,59746028490,2016-01-18,0.1813,PANTOPRAZOLE TAB 40MG,ULCER DRUGS,FL,Ulcer,17106797,WALGREENS #6611,33603


In [89]:
basket = (df
          .groupby(['person_code', 'drug_name'])['id']
          .count().unstack().reset_index().fillna(0)
          .set_index('person_code'))

basket.head(10)

# basket[basket['person_code']=='13711760']

drug_name,"1ML SYRINGE MIS 28GX1/2""","1ML TB SYRNG MIS 25GX5/8""","1ML TB SYRNG MIS 27GX1/2""","3ML LL SYRNG MIS 18GX1.5""","3ML LL SYRNG MIS 20GX1""","3ML LL SYRNG MIS 21GX1.5""","3ML LL SYRNG MIS 22GX1""","3ML LL SYRNG MIS 22GX1.5""","3ML LL SYRNG MIS 23GX1""","3ML LL SYRNG MIS 23GX1.5""",...,ZOVIA 1/35E TAB,ZOVIRAX CRE 5%,ZUBSOLV SUB 11.4-2.9,ZUBSOLV SUB 5.7-1.4,ZYCLARA PUMP CRE 3.75%,ZYLET SUS 0.5-0.3%,ZYRTEC,ZYRTEC ALLGY TAB 10MG,ZYTIGA TAB 500MG,ZYVOX TAB 600MG
person_code,,,,,,,,,,,,,,,,,,,,,
13132450,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13133262,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13133575,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13133736,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13133739,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13133910,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13133962,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13133971,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13133981,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [88]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
display(basket_sets.shape)
basket_sets.head()

(9220, 3191)

drug_name,"1ML SYRINGE MIS 28GX1/2""","1ML TB SYRNG MIS 25GX5/8""","1ML TB SYRNG MIS 27GX1/2""","3ML LL SYRNG MIS 18GX1.5""","3ML LL SYRNG MIS 20GX1""","3ML LL SYRNG MIS 21GX1.5""","3ML LL SYRNG MIS 22GX1""","3ML LL SYRNG MIS 22GX1.5""","3ML LL SYRNG MIS 23GX1""","3ML LL SYRNG MIS 23GX1.5""",...,ZOVIA 1/35E TAB,ZOVIRAX CRE 5%,ZUBSOLV SUB 11.4-2.9,ZUBSOLV SUB 5.7-1.4,ZYCLARA PUMP CRE 3.75%,ZYLET SUS 0.5-0.3%,ZYRTEC,ZYRTEC ALLGY TAB 10MG,ZYTIGA TAB 500MG,ZYVOX TAB 600MG
person_code,,,,,,,,,,,,,,,,,,,,,
13132450,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13133262,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13133575,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13133736,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13133739,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [86]:
frequent_itemsets = apriori(basket_sets, min_support=0.02, use_colnames=True)
# frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))

# frequent_itemsets = frequent_itemsets[(frequent_itemsets['length'] == 2)]
frequent_itemsets

,support,itemsets
0,0.026030,(AMLODIPINE TAB 5MG)
1,0.110087,(AMOX/K CLAV TAB 875-125)
2,0.099675,(AMOXICILLIN CAP 500MG)
3,0.059978,(AMOXICILLIN SUS 400/5ML)
4,0.053579,(AMOXICILLIN TAB 875MG)
5,0.039154,(APAP/CODEINE TAB 300-30MG)
6,0.026464,(ATORVASTATIN TAB 10MG)
7,0.028850,(ATORVASTATIN TAB 20MG)
8,0.020390,(ATORVASTATIN TAB 40MG)
9,0.020390,(AZITHROMYCIN SUS 200/5ML)


In [87]:
rules = association_rules(frequent_itemsets, metric="lift")
display(rules.shape)
rules

(22, 9)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(AMOX/K CLAV TAB 875-125),(AZITHROMYCIN TAB 250MG),0.110087,0.150108,0.029718,0.269951,1.798371,0.013193,1.164156
1,(AZITHROMYCIN TAB 250MG),(AMOX/K CLAV TAB 875-125),0.150108,0.110087,0.029718,0.197977,1.798371,0.013193,1.109586
2,(AMOX/K CLAV TAB 875-125),(FLUTICASONE SPR 50MCG),0.110087,0.086551,0.024403,0.221675,2.561206,0.014875,1.173609
3,(FLUTICASONE SPR 50MCG),(AMOX/K CLAV TAB 875-125),0.086551,0.110087,0.024403,0.281955,2.561206,0.014875,1.239356
4,(METHYLPRED TAB 4MG),(AMOX/K CLAV TAB 875-125),0.085900,0.110087,0.022017,0.256313,2.328283,0.012561,1.196624
5,(AMOX/K CLAV TAB 875-125),(METHYLPRED TAB 4MG),0.110087,0.085900,0.022017,0.200000,2.328283,0.012561,1.142625
6,(AMOX/K CLAV TAB 875-125),(PREDNISONE TAB 20MG),0.110087,0.078091,0.020065,0.182266,2.334018,0.011468,1.127395
7,(PREDNISONE TAB 20MG),(AMOX/K CLAV TAB 875-125),0.078091,0.110087,0.020065,0.256944,2.334018,0.011468,1.197640
8,(AMOXICILLIN CAP 500MG),(AZITHROMYCIN TAB 250MG),0.099675,0.150108,0.021041,0.211099,1.406310,0.006079,1.077311
9,(AZITHROMYCIN TAB 250MG),(AMOXICILLIN CAP 500MG),0.150108,0.099675,0.021041,0.140173,1.406310,0.006079,1.047101


In [38]:
rules[ (rules['lift'] >= 6) &
       (rules['confidence'] >= 0.8) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


In [91]:
basket_i = (df
          .groupby(['person_code', 'drug_indication'])['id']
          .count().unstack().reset_index().fillna(0)
          .set_index('person_code'))

basket_i.head(10)

drug_indication,ADHD,Acne Products,Addison's disease,Adjuvant Agents,Alcohol Abuse,Allergies,Alternative Medicines,Anemia,Anticoagulation,Antidotes,...,Ulcer,Urinary infection,Vaccines,Vaginal Infection,Vasomotor Symptoms,Vitamins,Warts,Weight Loss,Wound Treatment,\N
person_code,,,,,,,,,,,,,,,,,,,,,
13132450,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13133262,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13133575,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,...,15.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,3.0
13133736,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
13133739,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13133910,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13133962,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
13133971,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13133981,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [92]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets_i = basket_i.applymap(encode_units)
display(basket_sets_i.shape)
basket_sets_i.head()

(9220, 127)

drug_indication,ADHD,Acne Products,Addison's disease,Adjuvant Agents,Alcohol Abuse,Allergies,Alternative Medicines,Anemia,Anticoagulation,Antidotes,...,Ulcer,Urinary infection,Vaccines,Vaginal Infection,Vasomotor Symptoms,Vitamins,Warts,Weight Loss,Wound Treatment,\N
person_code,,,,,,,,,,,,,,,,,,,,,
13132450,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13133262,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13133575,0,0,0,0,0,1,0,0,0,0,...,1,0,0,1,0,1,0,0,0,1
13133736,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
13133739,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [96]:
frequent_itemsets_i = apriori(basket_sets_i, min_support=0.08, use_colnames=True)
# frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
# frequent_itemsets = frequent_itemsets[(frequent_itemsets['length'] == 2)]
frequent_itemsets_i

,support,itemsets
0,0.108134,(Anxiety)
1,0.159761,(Asthma)
2,0.590672,(Bacterial Infection)
3,0.105098,(Cold)
4,0.090889,(Contraception)
5,0.137310,(Cough/Cold/Allergy)
6,0.183514,(Depression)
7,0.157158,(High Cholesterol)
8,0.224620,(Hypertension)
9,0.218655,(Inflammation)


In [98]:
rules_i = association_rules(frequent_itemsets_i, metric="lift")
display(rules_i.shape)
rules_i.sort_values(['support','confidence', 'lift'], ascending=[False, False, False])

(28, 9)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
18,(Pain - Opioid),(Bacterial Infection),0.232430,0.590672,0.181128,0.779281,1.319312,0.043838,1.854522
19,(Bacterial Infection),(Pain - Opioid),0.590672,0.232430,0.181128,0.306647,1.319312,0.043838,1.107042
12,(Inflammation),(Bacterial Infection),0.218655,0.590672,0.173970,0.795635,1.346999,0.044816,2.002923
13,(Bacterial Infection),(Inflammation),0.590672,0.218655,0.173970,0.294528,1.346999,0.044816,1.107549
15,(Inflammation - NSAID),(Bacterial Infection),0.193167,0.590672,0.138178,0.715328,1.211041,0.024079,1.437894
14,(Bacterial Infection),(Inflammation - NSAID),0.590672,0.193167,0.138178,0.233933,1.211041,0.024079,1.053215
10,(Hypertension),(Bacterial Infection),0.224620,0.590672,0.128525,0.572187,0.968705,-0.004152,0.956792
11,(Bacterial Infection),(Hypertension),0.590672,0.224620,0.128525,0.217591,0.968705,-0.004152,0.991016
5,(Cough/Cold/Allergy),(Bacterial Infection),0.137310,0.590672,0.121475,0.884676,1.497744,0.040370,3.549374
4,(Bacterial Infection),(Cough/Cold/Allergy),0.590672,0.137310,0.121475,0.205656,1.497744,0.040370,1.086040
